In [1]:
import xgboost as xgb
from sklearn.mixture import GaussianMixture

## load trained algorithms

### load SAVIC-P - C$\alpha$ XGB classifier 

In [2]:
xgbc_sus_ca = xgb.XGBClassifier()
xgbc_sus_ca.load_model('Output/ML/models/xgbc_sus_ca.json')

### load SAVIC-Q - C$\alpha$ XGB classifier \& regressor

In [3]:
xgbc_kca = xgb.XGBClassifier()
xgbc_kca.load_model('Output/ML/models/xgbc_kca.json')

xgbr_c1_a1_k1 = xgb.XGBRegressor()
xgbr_c1_a1_k1.load_model('Output/ML/models/xgbr_ca_c1_a1_k1.json')

xgbr_c1_a1_k0 = xgb.XGBRegressor()
xgbr_c1_a1_k0.load_model('Output/ML/models/xgbr_ca_c1_a1_k0.json')

xgbr_c1_a0 = xgb.XGBRegressor()
xgbr_c1_a0.load_model('Output/ML/models/xgbr_ca_c1_a0_k0.json')

xgbr_c0_a1 = xgb.XGBRegressor()
xgbr_c0_a1.load_model('Output/ML/models/xgbr_ca_c0_a1_k0.json')

### load SAVIC-C - C$\alpha$ GM Classifier

In [4]:
gmm_ca_name = 'Output/ML/models/GMM_CA/GMM_CA'
means = np.load(gmm_ca_name + '_means.npy')
covar = np.load(gmm_ca_name + '_covariances.npy')
gmm_ca = GaussianMixture(n_components = len(means), covariance_type='full')
gmm_ca.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
gmm_ca.weights_ = np.load(gmm_ca_name + '_weights.npy')
gmm_ca.means_ = means
gmm_ca.covariances_ = covar

## SAVIC-P - core \& $\alpha$

### load SAVIC-P - C$\alpha$ processing function

In [5]:
def SAVIC_P_CA(df_ca_):
    df_ca_pre_SP_ = df_ca_.copy()
    df_ca_pre_SP_[['beta_par_core', 'alph_c', 'tau_a', 'alph_a']] = \
    np.log10(df_ca_pre_SP_[['beta_par_core', 'alph_c', 'tau_a', 'alph_a']])
    df_ca_pre_SP_['unstable'] = xgbc_sus_ca.predict(df_ca_pre_SP_.values).astype('bool')
    
    df_ca_post_SP_ = df_ca_pre_SP_.copy()
    df_ca_post_SP_[['beta_par_core', 'alph_c', 'tau_a', 'alph_a']] = \
    pow(10., df_ca_post_SP_[['beta_par_core', 'alph_c', 'tau_a', 'alph_a']])
    
    return df_ca_post_SP_

### run SAVIC-P - C$\alpha$ 

In [6]:
df_ca_post_SP = SAVIC_P_CA(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-P_CA_pre_P'))
df_ca_post_SP

,beta_par_core,alph_c,tau_a,alph_a,D_a,vv_a,unstable
0,1.7,0.5,0.5,0.7,0.02,0.07,True
1,0.6,1.0,0.4,1.9,0.03,0.06,True
2,0.5,0.7,0.8,0.8,0.01,0.00,False
3,3.6,0.4,0.6,1.0,0.02,0.15,True
4,1.0,1.0,0.5,1.2,0.09,0.20,False
5,0.2,2.4,0.2,0.9,0.03,0.15,True


## SAVIC-Q - core \& $\alpha$

### load SAVIC-Q - C$\alpha$ processing function

In [7]:
def SAVIC_Q_CA(df_ca_):
    
    df_ca_uns_ = df_ca_[df_ca_['unstable']].drop(columns = ['unstable'])
    
    df_ca_uns_pre_SQ_ = df_ca_uns_.copy()
    df_ca_uns_pre_SQ_[['beta_par_core', 'alph_c', 'alph_a']] = \
    np.log10(df_ca_uns_pre_SQ_[['beta_par_core', 'alph_c', 'alph_a']])
    df_ca_uns_pre_SQ_.columns = [['log_beta_par_core', 'log_alph_c', 'tau_a', 'log_alph_a', 'D_a', 'vv_a']]
    
    df_ca_uns_postC_SQ_ = df_ca_uns_pre_SQ_.copy()
    df_ca_uns_postC_SQ_['group'] = xgbc_kca.predict(df_ca_uns_pre_SQ_.values)
    
    df_ca_uns_preQ_SQ_ = df_ca_uns_postC_SQ_.copy()
    df_ca_uns_preQ_SQ_[['log_beta_par_core', 'log_alph_c', 'log_alph_a']] = \
    pow(10., df_ca_uns_preQ_SQ_[['log_beta_par_core', 'log_alph_c', 'log_alph_a']]) 
    df_ca_uns_preQ_SQ_.columns = ['beta_par_core', 'alph_c', 'tau_a', 'alph_a', 'D_a', 'vv_a', 'group']
    
    df_ca_uns_preQ_SQ_0_ = df_ca_uns_preQ_SQ_[df_ca_uns_preQ_SQ_['group'] == 0].drop(columns = ['group'])
    df_ca_uns_preQ_SQ_1_ = df_ca_uns_preQ_SQ_[df_ca_uns_preQ_SQ_['group'] == 1].drop(columns = ['group'])
    df_ca_uns_preQ_SQ_2_ = df_ca_uns_preQ_SQ_[df_ca_uns_preQ_SQ_['group'] == 2].drop(columns = ['group'])
    df_ca_uns_preQ_SQ_3_ = df_ca_uns_preQ_SQ_[df_ca_uns_preQ_SQ_['group'] == 3].drop(columns = ['group'])
    df_ca_uns_preQ_SQ_4_ = df_ca_uns_preQ_SQ_[df_ca_uns_preQ_SQ_['group'] == 4].drop(columns = ['group'])
    df_ca_uns_preQ_SQ_5_ = df_ca_uns_preQ_SQ_[df_ca_uns_preQ_SQ_['group'] == 5].drop(columns = ['group'])
    
    # regressors
    if df_ca_uns_preQ_SQ_0_.size > 0:
        df_ca_uns_preQ_SQ_0_log_ = df_ca_uns_preQ_SQ_0_.copy()


        df_ca_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_a']] = np.log10(df_ca_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_a']])
        df_ca_uns_preQ_SQ_0_log_[['Pow_core', 'Pow_alpha', 'kB_angle']] = xgbr_c1_a1_k1.predict(df_ca_uns_preQ_SQ_0_log_).astype(float)

        df_ca_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'Pow_alpha', 'kB_angle']] = \
        10**df_ca_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]

        df_ca_uns_postQ_SQ_0_ = df_ca_uns_preQ_SQ_0_log_.copy()
        df_ca_uns_postQ_SQ_0_ = df_ca_uns_postQ_SQ_0_[['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]
    else:
        df_ca_uns_postQ_SQ_0_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle'])

    if df_ca_uns_preQ_SQ_1_.size > 0:
        df_ca_uns_preQ_SQ_1_log_ = df_ca_uns_preQ_SQ_1_.copy()


        df_ca_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_a']] = np.log10(df_ca_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_a']])
        df_ca_uns_preQ_SQ_1_log_[['Pow_core', 'Pow_alpha', 'kB_angle']] = xgbr_c1_a1_k0.predict(df_ca_uns_preQ_SQ_1_log_).astype(float)

        df_ca_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'Pow_alpha', 'kB_angle']] = \
        10**df_ca_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]

        df_ca_uns_postQ_SQ_1_ = df_ca_uns_preQ_SQ_1_log_.copy()
        df_ca_uns_postQ_SQ_1_ = df_ca_uns_postQ_SQ_1_[['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]
    else:
        df_ca_uns_postQ_SQ_1_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle'])

    if df_ca_uns_preQ_SQ_2_.size > 0:
        df_ca_uns_preQ_SQ_2_log_ = df_ca_uns_preQ_SQ_2_.copy()

        df_ca_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_a']] = np.log10(df_ca_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_a']])
        df_ca_uns_preQ_SQ_2_log_[['Pow_core', 'kB_angle']] = xgbr_c1_a0.predict(df_ca_uns_preQ_SQ_2_log_).astype(float)

        df_ca_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'kB_angle']] = \
        10**df_ca_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'kB_angle']]

        df_ca_uns_postQ_SQ_2_ = df_ca_uns_preQ_SQ_2_log_.copy()
        df_ca_uns_postQ_SQ_2_['Pow_alpha'] = 0
        df_ca_uns_postQ_SQ_2_ = df_ca_uns_postQ_SQ_2_[['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]
    else:
        df_ca_uns_postQ_SQ_2_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle'])

    if df_ca_uns_preQ_SQ_3_.size > 0:
        df_ca_uns_preQ_SQ_3_log_ = df_ca_uns_preQ_SQ_3_.copy()


        df_ca_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_a']] = np.log10(df_ca_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_a']])
        df_ca_uns_preQ_SQ_3_log_[['Pow_core', 'kB_angle']] = xgbr_c1_a0.predict(df_ca_uns_preQ_SQ_3_log_).astype(float)

        df_ca_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'kB_angle']] = \
        10**df_ca_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'kB_angle']]

        df_ca_uns_postQ_SQ_3_ = df_ca_uns_preQ_SQ_3_log_.copy()
        df_ca_uns_postQ_SQ_3_['Pow_alpha'] = 0
        df_ca_uns_postQ_SQ_3_ = df_ca_uns_postQ_SQ_3_[['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]
    else:
        df_ca_uns_postQ_SQ_3_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle'])

    if df_ca_uns_preQ_SQ_4_.size > 0:
        df_ca_uns_preQ_SQ_4_log_ = df_ca_uns_preQ_SQ_4_.copy()


        df_ca_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_a']] = np.log10(df_ca_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_a']])
        df_ca_uns_preQ_SQ_4_log_[['Pow_alpha', 'kB_angle']] = xgbr_c0_a1.predict(df_ca_uns_preQ_SQ_4_log_).astype(float)

        df_ca_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_alpha', 'kB_angle']] = \
        10**df_ca_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_alpha', 'kB_angle']]

        df_ca_uns_postQ_SQ_4_ = df_ca_uns_preQ_SQ_4_log_.copy()
        df_ca_uns_postQ_SQ_4_['Pow_core'] = 0
        df_ca_uns_postQ_SQ_4_ = df_ca_uns_postQ_SQ_4_[['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]
    else:
        df_ca_uns_postQ_SQ_4_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle'])

    if df_ca_uns_preQ_SQ_5_.size > 0:
        df_ca_uns_preQ_SQ_5_log_ = df_ca_uns_preQ_SQ_5_.copy()


        df_ca_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_a']] = np.log10(df_ca_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_a']])
        df_ca_uns_preQ_SQ_5_log_[['Pow_alpha', 'kB_angle']] = xgbr_c0_a1.predict(df_ca_uns_preQ_SQ_5_log_).astype(float)

        df_ca_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_alpha', 'kB_angle']] = \
        10**df_ca_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_alpha', 'kB_angle']]

        df_ca_uns_postQ_SQ_5_ = df_ca_uns_preQ_SQ_5_log_.copy()
        df_ca_uns_postQ_SQ_5_['Pow_core'] = 0
        df_ca_uns_postQ_SQ_5_ = df_ca_uns_postQ_SQ_5_[['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]
    else:
        df_ca_uns_postQ_SQ_5_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle'])

    df_ca_uns_postQ_SQ_ = pd.concat([df_ca_uns_postQ_SQ_0_, df_ca_uns_postQ_SQ_1_, df_ca_uns_postQ_SQ_2_, \
                                 df_ca_uns_postQ_SQ_3_, df_ca_uns_postQ_SQ_4_, df_ca_uns_postQ_SQ_5_]).sort_index()
    
    df_ca_postQ_SQ_ = df_ca_.copy()

    df_ca_postQ_SQ_['group'] = np.nan
    df_ca_postQ_SQ_.loc[df_ca_uns_preQ_SQ_.index, 'group' ] = \
    df_ca_uns_preQ_SQ_['group']
    df_ca_postQ_SQ_['group'] = df_ca_postQ_SQ_['group']#.astype('Int64')

    df_ca_postQ_SQ_[['Pow_core', 'Pow_alpha', 'kB_angle']] = np.nan
    df_ca_postQ_SQ_.loc[df_ca_uns_postQ_SQ_.index, ['Pow_core', 'Pow_alpha', 'kB_angle'] ] = \
    df_ca_uns_postQ_SQ_[['Pow_core', 'Pow_alpha', 'kB_angle'] ]
    
    return df_ca_postQ_SQ_

### run SAVIC-Q - C$\alpha$ 

In [8]:
df_ca_post_SQ = SAVIC_Q_CA(df_ca_post_SP)
df_ca_post_SQ

,beta_par_core,alph_c,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_alpha,kB_angle
0,1.7,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004490,0.000000,0.001962
1,0.6,1.0,0.4,1.9,0.03,0.06,True,5.0,0.000000,0.007504,0.004290
2,0.5,0.7,0.8,0.8,0.01,0.00,False,NaN,NaN,NaN,NaN
3,3.6,0.4,0.6,1.0,0.02,0.15,True,2.0,0.346296,0.000000,0.002381
4,1.0,1.0,0.5,1.2,0.09,0.20,False,NaN,NaN,NaN,NaN
5,0.2,2.4,0.2,0.9,0.03,0.15,True,3.0,0.030773,0.000000,0.002031


## SAVIC-C - core \& $\alpha$

### load SAVIC-C - C$\alpha$ processing function

In [9]:
def SAVIC_C_CA(df_ca_):
    
    df_ca_uns_ = df_ca_[df_ca_['unstable']].drop(columns = ['unstable'])
    
    df_ca_uns_pre_SC_ = df_ca_uns_.copy()

    df_ca_uns_pre_SC_['uns_IC'] = (1. + 0.367 / ( df_ca_uns_pre_SC_['beta_par_core'] - 0.011 )**0.364) < df_ca_uns_pre_SC_['alph_c']
    df_ca_uns_pre_SC_['uns_Mirror'] = (1. + 0.702 / ( df_ca_uns_pre_SC_['beta_par_core'] + 0.009 )**0.674) < df_ca_uns_pre_SC_['alph_c']
    df_ca_uns_pre_SC_['uns_FM'] = (1. - 0.408 / ( df_ca_uns_pre_SC_['beta_par_core'] - 0.410 )**0.529) > df_ca_uns_pre_SC_['alph_c']
    df_ca_uns_pre_SC_['uns_OFH'] = (1. - 1.454 / ( df_ca_uns_pre_SC_['beta_par_core'] + 0.178 )**1.023) > df_ca_uns_pre_SC_['alph_c']
    df_ca_uns_pre_SC_['uns_core'] = df_ca_uns_pre_SC_['uns_OFH'] | df_ca_uns_pre_SC_['uns_FM'] | df_ca_uns_pre_SC_['uns_Mirror'] | df_ca_uns_pre_SC_['uns_IC']

    df_ca_uns_pre_SC_['beta_par_alpha'] = df_ca_uns_pre_SC_['beta_par_core'] * df_ca_uns_pre_SC_['D_a'] / df_ca_uns_pre_SC_['tau_a']
    df_ca_uns_pre_SC_['lambda_FH_beta'] = ( (df_ca_uns_pre_SC_['beta_par_core'] + df_ca_uns_pre_SC_['beta_par_alpha']) - \
    (df_ca_uns_pre_SC_['beta_par_core'] * df_ca_uns_pre_SC_['alph_c'] + df_ca_uns_pre_SC_['beta_par_alpha'] * df_ca_uns_pre_SC_['alph_a']) ) / 2.
    df_ca_uns_pre_SC_['lambda_FH_alpha'] = 4 * df_ca_uns_pre_SC_['D_a'] / (1. + df_ca_uns_pre_SC_['D_a'] + 4 * df_ca_uns_pre_SC_['D_a']) * df_ca_uns_pre_SC_['vv_a']**2.
    df_ca_uns_pre_SC_['lambda_FH'] = df_ca_uns_pre_SC_['lambda_FH_beta'] + df_ca_uns_pre_SC_['lambda_FH_alpha']
    df_ca_uns_pre_SC_['lambda_FH'] = df_ca_uns_pre_SC_['lambda_FH'] > 1.

    df_ca_uns_pre_SC_['Pow_core'][df_ca_uns_pre_SC_['Pow_core'] < 0] = 0
    df_ca_uns_pre_SC_['Pow_alpha'][df_ca_uns_pre_SC_['Pow_alpha'] < 0] = 0
    
    column_list_ca_ = ['beta_par_core', 'alph_c', 'kB_angle', \
                       'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', \
                       'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core', 'lambda_FH']

    df_ca_uns_pre_SC_cut_ = df_ca_uns_pre_SC_[column_list_ca_]
    
    df_ca_uns_pre_SC_cut_[['beta_par_core', 'alph_c']] = np.log10(df_ca_uns_pre_SC_cut_[['beta_par_core', 'alph_c']])
    df_ca_uns_pre_SC_cut_.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
    
    df_ca_uns_post_SC_ = df_ca_uns_pre_SC_cut_.copy()
    df_ca_uns_post_SC_['cluster'] = gmm_ca.predict(df_ca_uns_pre_SC_cut_.values)
    
    df_ca_uns_post_SC_['ins_type'] = np.nan
    ins_types_ca_ = ['Parallel Firehose', 'IC (A)', 'Oblique Firehose', \
                     'CGL Firehose; Mirror', 'A anis; borderline PFH', 'IC (C)']
    for cluster, ins_type in zip(np.arange(6), ins_types_ca_):
        df_ca_uns_post_SC_['ins_type'][df_ca_uns_post_SC_['cluster'] == cluster] = ins_type
    
    df_ca_post_SC_ = df_ca_.copy()
    df_ca_post_SC_['ins_type'] = np.nan
    df_ca_post_SC_.loc[df_ca_uns_post_SC_.index, ['ins_type']] = df_ca_uns_post_SC_['ins_type']
    
    return df_ca_post_SC_

### run SAVIC-C - C$\alpha$ 

In [10]:
df_ca_post_SP = SAVIC_C_CA(df_ca_post_SQ)
df_ca_post_SP

,beta_par_core,alph_c,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_alpha,kB_angle,ins_type
0,1.7,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004490,0.000000,0.001962,Parallel Firehose
1,0.6,1.0,0.4,1.9,0.03,0.06,True,5.0,0.000000,0.007504,0.004290,IC (A)
2,0.5,0.7,0.8,0.8,0.01,0.00,False,NaN,NaN,NaN,NaN,NaN
3,3.6,0.4,0.6,1.0,0.02,0.15,True,2.0,0.346296,0.000000,0.002381,CGL Firehose; Mirror
4,1.0,1.0,0.5,1.2,0.09,0.20,False,NaN,NaN,NaN,NaN,NaN
5,0.2,2.4,0.2,0.9,0.03,0.15,True,3.0,0.030773,0.000000,0.002031,IC (C)


## SAVIC - C$\alpha$

### load SAVIC - C$\alpha$ processing function

In [11]:
def SAVIC_CoreAlpha(df_ca_):
    
    df_ca_post_SP_ = SAVIC_P_CA(df_ca_)
    df_ca_post_SQ_ = SAVIC_Q_CA(df_ca_post_SP_)
    df_ca_post_SC_ = SAVIC_C_CA(df_ca_post_SQ_)
    
    return df_ca_post_SC_

### run SAVIC - C$\alpha$

In [12]:
df_ca_post_SC = SAVIC_CoreAlpha(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-P_CA_pre_P'))
df_ca_post_SC

,beta_par_core,alph_c,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_alpha,kB_angle,ins_type
0,1.7,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004490,0.000000,0.001962,Parallel Firehose
1,0.6,1.0,0.4,1.9,0.03,0.06,True,5.0,0.000000,0.007504,0.004290,IC (A)
2,0.5,0.7,0.8,0.8,0.01,0.00,False,NaN,NaN,NaN,NaN,NaN
3,3.6,0.4,0.6,1.0,0.02,0.15,True,2.0,0.346296,0.000000,0.002381,CGL Firehose; Mirror
4,1.0,1.0,0.5,1.2,0.09,0.20,False,NaN,NaN,NaN,NaN,NaN
5,0.2,2.4,0.2,0.9,0.03,0.15,True,3.0,0.030773,0.000000,0.002031,IC (C)
